In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import os
import uuid
import safe
import transformers
import evaluate
from dataclasses import dataclass, field
from loguru import logger
from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import set_seed
from transformers.trainer_utils import get_last_checkpoint
from transformers import TrainingArguments
from safe.trainer.model import SAFEDoubleHeadsModel
from safe.tokenizer import SAFETokenizer
from safe.trainer.data_utils import get_dataset
from safe.trainer.collator import SAFECollator
from safe.trainer.trainer_utils import SAFETrainer
from safe.trainer.cli import ModelArguments
from safe.trainer.cli import DataArguments
from safe.trainer.cli import train


[2023-08-15 12:38:48,928] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [3]:
dataset = "../data/"
tokenizer = "../tokenizer/tokenizer-custom.json"

In [4]:
#dt = get_dataset(dataset, streaming=True, tokenize_column="input", property_column=None)

In [5]:
model_args_1 = ModelArguments(config=None, tokenizer=tokenizer, prop_loss_coeff=1e-2, num_labels=18, include_descriptors=False, wandb_watch="gradient")
model_args_2 = ModelArguments(config=None, tokenizer=tokenizer, prop_loss_coeff=1e-2, num_labels=18, include_descriptors=True, wandb_watch="gradient")
data_args = DataArguments(dataset=dataset,  streaming=True, text_column="input", is_tokenized=False, property_column=None,  max_eval_samples=1000)

In [6]:
# params
ddp = False
gradient_accumulation_steps = 2

batch_size = 32
warmup_steps = 10
num_epochs = 1
learning_rate = 1e-5

logging_steps = 10
output_dir = "../output/"
max_steps = 50
save_steps = 25
cache_dir = None
eval_accumulation_steps = 100 # uses this to avoid 
eval_delay=None

training_args = TrainingArguments(
        do_train=True, 
        do_eval=True, 
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=warmup_steps,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        logging_steps=logging_steps,
        #eval_steps=0.5,
        optim="adamw_torch",
        output_dir=output_dir,
        report_to="wandb",
        save_safetensors=True,
        torch_compile=True,
        max_steps=max_steps,
        log_level="warning",
        label_names=["labels", "mc_labels"],
        save_steps=save_steps,
        eval_delay=eval_delay,
        eval_accumulation_steps=eval_accumulation_steps

)

In [7]:
! rm -rf ../output

In [9]:
import wandb
wandb.finish()
train(model_args_2, data_args, training_args)

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train/epoch,▁▃▅▆██
train/global_step,▁▃▅▆██
train/learning_rate,█▆▅▃▁
train/loss,█▅▆▃▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,1.0
train/global_step,50


[WARNING|trainer.py:2596] 2023-08-15 12:41:01,118 >> There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


/home/emmanuel_invivoai_com/mambaforge/envs/safe-space/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


***** train metrics *****
  epoch                    =       1.02
  total_flos               =   488961GF
  train_loss               =     1.8506
  train_runtime            = 0:00:07.80
  train_samples_per_second =     819.47
  train_steps_per_second   =      6.402
***** eval metrics *****
  epoch                   =                   1.02
  eval_accuracy           =                 0.6741
  eval_loss               =                99.9082
  eval_mse                =              9646.5236
  eval_runtime            =             0:00:04.93
  eval_samples_per_second =                202.537
  eval_steps_per_second   =                  12.76
  perplexity              = 2.4522313125199866e+43
